# MAP Type - Associative Arrays

This notebook demonstrates the **MAP type** in elastic-script - a key-value data structure for associative arrays.

## Features

- **MAP literals**: `MAP { 'key' => value, ... }`
- **Key-value storage**: Store any type as values, string/number keys
- **Immutable operations**: Functions return new maps (functional style)
- **12 built-in functions**: GET, PUT, REMOVE, KEYS, VALUES, SIZE, and more

## 1. Creating Maps

In [ ]:
-- Create an empty map
CREATE PROCEDURE test_empty_map()
BEGIN
    DECLARE empty_map MAP;
    SET empty_map = MAP {};
    PRINT 'Empty map size: ' || MAP_SIZE(empty_map);
END PROCEDURE;

CALL test_empty_map();

In [ ]:
-- Create a map with initial values
CREATE PROCEDURE test_map_literal()
BEGIN
    DECLARE person MAP = MAP {
        'name' => 'Alice',
        'age' => 30,
        'active' => true
    };
    
    PRINT 'Person name: ' || MAP_GET(person, 'name');
    PRINT 'Person age: ' || MAP_GET(person, 'age');
    PRINT 'Is active: ' || MAP_GET(person, 'active');
END PROCEDURE;

CALL test_map_literal();

## 2. MAP Functions

In [ ]:
-- MAP_GET and MAP_GET_OR_DEFAULT
CREATE PROCEDURE test_map_get()
BEGIN
    DECLARE config MAP = MAP {
        'host' => 'localhost',
        'port' => 9200
    };
    
    -- Get existing key
    PRINT 'Host: ' || MAP_GET(config, 'host');
    
    -- Get with default for missing key
    PRINT 'Timeout: ' || MAP_GET_OR_DEFAULT(config, 'timeout', 30);
END PROCEDURE;

CALL test_map_get();

In [ ]:
-- MAP_PUT - returns new map with added/updated key
CREATE PROCEDURE test_map_put()
BEGIN
    DECLARE original MAP = MAP { 'a' => 1, 'b' => 2 };
    
    -- Add new key
    DECLARE updated MAP;
    SET updated = MAP_PUT(original, 'c', 3);
    
    PRINT 'Original size: ' || MAP_SIZE(original);
    PRINT 'Updated size: ' || MAP_SIZE(updated);
    PRINT 'New key value: ' || MAP_GET(updated, 'c');
END PROCEDURE;

CALL test_map_put();

In [ ]:
-- MAP_REMOVE - returns new map without specified key
CREATE PROCEDURE test_map_remove()
BEGIN
    DECLARE data MAP = MAP { 'keep' => 1, 'remove' => 2, 'also_keep' => 3 };
    
    DECLARE cleaned MAP;
    SET cleaned = MAP_REMOVE(data, 'remove');
    
    PRINT 'Original keys: ' || ARRAY_JOIN(MAP_KEYS(data), ', ');
    PRINT 'After removal: ' || ARRAY_JOIN(MAP_KEYS(cleaned), ', ');
END PROCEDURE;

CALL test_map_remove();

In [ ]:
-- MAP_KEYS and MAP_VALUES
CREATE PROCEDURE test_keys_values()
BEGIN
    DECLARE scores MAP = MAP {
        'alice' => 95,
        'bob' => 87,
        'charlie' => 92
    };
    
    PRINT 'All students: ' || ARRAY_JOIN(MAP_KEYS(scores), ', ');
    PRINT 'All scores: ' || ARRAY_JOIN(MAP_VALUES(scores), ', ');
    PRINT 'Total students: ' || MAP_SIZE(scores);
END PROCEDURE;

CALL test_keys_values();

In [ ]:
-- MAP_CONTAINS_KEY and MAP_CONTAINS_VALUE
CREATE PROCEDURE test_contains()
BEGIN
    DECLARE inventory MAP = MAP {
        'apples' => 50,
        'oranges' => 30,
        'bananas' => 0
    };
    
    -- Check for key
    IF MAP_CONTAINS_KEY(inventory, 'apples') THEN
        PRINT 'We have apples!';
    END IF;
    
    -- Check for value
    IF MAP_CONTAINS_VALUE(inventory, 0) THEN
        PRINT 'Warning: Some items are out of stock';
    END IF;
END PROCEDURE;

CALL test_contains();

## 3. Advanced Operations

In [ ]:
-- MAP_MERGE - combine two maps
CREATE PROCEDURE test_map_merge()
BEGIN
    DECLARE defaults MAP = MAP {
        'timeout' => 30,
        'retries' => 3,
        'debug' => false
    };
    
    DECLARE overrides MAP = MAP {
        'timeout' => 60,
        'debug' => true
    };
    
    -- Merge: overrides take precedence
    DECLARE config MAP;
    SET config = MAP_MERGE(defaults, overrides);
    
    PRINT 'Timeout: ' || MAP_GET(config, 'timeout');  -- 60 (overridden)
    PRINT 'Retries: ' || MAP_GET(config, 'retries');  -- 3 (from defaults)
    PRINT 'Debug: ' || MAP_GET(config, 'debug');      -- true (overridden)
END PROCEDURE;

CALL test_map_merge();

In [ ]:
-- MAP_FROM_ARRAYS - create map from parallel arrays
CREATE PROCEDURE test_from_arrays()
BEGIN
    DECLARE names ARRAY = ['alice', 'bob', 'charlie'];
    DECLARE ages ARRAY = [25, 30, 35];
    
    DECLARE age_map MAP;
    SET age_map = MAP_FROM_ARRAYS(names, ages);
    
    PRINT 'Alice age: ' || MAP_GET(age_map, 'alice');
    PRINT 'Bob age: ' || MAP_GET(age_map, 'bob');
    PRINT 'Charlie age: ' || MAP_GET(age_map, 'charlie');
END PROCEDURE;

CALL test_from_arrays();

In [ ]:
-- MAP_ENTRIES - convert to array of {key, value} documents
CREATE PROCEDURE test_entries()
BEGIN
    DECLARE data MAP = MAP { 'x' => 10, 'y' => 20, 'z' => 30 };
    
    DECLARE entries ARRAY;
    SET entries = MAP_ENTRIES(data);
    
    PRINT 'Number of entries: ' || ARRAY_LENGTH(entries);
    
    -- Each entry is a document with 'key' and 'value' fields
    DECLARE i NUMBER;
    FOR i IN 1..ARRAY_LENGTH(entries) LOOP
        DECLARE entry DOCUMENT;
        SET entry = entries[i];
        PRINT 'Key: ' || entry['key'] || ', Value: ' || entry['value'];
    END LOOP;
END PROCEDURE;

CALL test_entries();

## 4. Practical Examples

In [ ]:
-- Example: Building a configuration object
CREATE PROCEDURE build_config()
BEGIN
    -- Start with defaults
    DECLARE config MAP = MAP {
        'host' => 'localhost',
        'port' => 9200,
        'ssl' => false,
        'timeout' => 30
    };
    
    -- Add authentication
    SET config = MAP_PUT(config, 'user', 'admin');
    SET config = MAP_PUT(config, 'auth_type', 'basic');
    
    -- Print final config
    DECLARE keys ARRAY;
    SET keys = MAP_KEYS(config);
    
    PRINT 'Configuration (' || MAP_SIZE(config) || ' settings):';
    DECLARE i NUMBER;
    FOR i IN 1..ARRAY_LENGTH(keys) LOOP
        DECLARE k STRING;
        SET k = keys[i];
        PRINT '  ' || k || ' = ' || MAP_GET(config, k);
    END LOOP;
END PROCEDURE;

CALL build_config();

In [ ]:
-- Example: Counting occurrences with a map
CREATE PROCEDURE word_count()
BEGIN
    DECLARE words ARRAY = ['apple', 'banana', 'apple', 'cherry', 'banana', 'apple'];
    DECLARE counts MAP = MAP {};
    
    DECLARE i NUMBER;
    FOR i IN 1..ARRAY_LENGTH(words) LOOP
        DECLARE word STRING;
        SET word = words[i];
        
        DECLARE current_count NUMBER;
        SET current_count = MAP_GET_OR_DEFAULT(counts, word, 0);
        SET counts = MAP_PUT(counts, word, current_count + 1);
    END LOOP;
    
    PRINT 'Word counts:';
    DECLARE keys ARRAY;
    SET keys = MAP_KEYS(counts);
    FOR i IN 1..ARRAY_LENGTH(keys) LOOP
        DECLARE k STRING;
        SET k = keys[i];
        PRINT '  ' || k || ': ' || MAP_GET(counts, k);
    END LOOP;
END PROCEDURE;

CALL word_count();

In [ ]:
-- Example: Nested maps for complex configuration
CREATE PROCEDURE nested_config()
BEGIN
    DECLARE config MAP = MAP {
        'database' => MAP {
            'host' => 'db.example.com',
            'port' => 5432,
            'name' => 'myapp'
        },
        'cache' => MAP {
            'enabled' => true,
            'ttl_seconds' => 3600
        }
    };
    
    -- Access nested values
    DECLARE db_config MAP;
    SET db_config = MAP_GET(config, 'database');
    
    PRINT 'Database host: ' || MAP_GET(db_config, 'host');
    PRINT 'Database port: ' || MAP_GET(db_config, 'port');
    
    DECLARE cache_config MAP;
    SET cache_config = MAP_GET(config, 'cache');
    PRINT 'Cache enabled: ' || MAP_GET(cache_config, 'enabled');
END PROCEDURE;

CALL nested_config();

## 5. MAP Functions Reference

| Function | Description | Example |
|----------|-------------|--------|
| `MAP_GET(map, key)` | Get value by key (null if missing) | `MAP_GET(m, 'name')` |
| `MAP_GET_OR_DEFAULT(map, key, default)` | Get value or default | `MAP_GET_OR_DEFAULT(m, 'port', 9200)` |
| `MAP_PUT(map, key, value)` | Return new map with key-value | `MAP_PUT(m, 'new', 1)` |
| `MAP_REMOVE(map, key)` | Return new map without key | `MAP_REMOVE(m, 'old')` |
| `MAP_KEYS(map)` | Get array of all keys | `MAP_KEYS(m)` |
| `MAP_VALUES(map)` | Get array of all values | `MAP_VALUES(m)` |
| `MAP_SIZE(map)` | Get number of entries | `MAP_SIZE(m)` |
| `MAP_CONTAINS_KEY(map, key)` | Check if key exists | `MAP_CONTAINS_KEY(m, 'x')` |
| `MAP_CONTAINS_VALUE(map, value)` | Check if value exists | `MAP_CONTAINS_VALUE(m, 42)` |
| `MAP_MERGE(map1, map2)` | Merge maps (map2 wins) | `MAP_MERGE(defaults, overrides)` |
| `MAP_FROM_ARRAYS(keys, values)` | Create from parallel arrays | `MAP_FROM_ARRAYS(['a','b'], [1,2])` |
| `MAP_ENTRIES(map)` | Convert to [{key, value}] array | `MAP_ENTRIES(m)` |